* Ссылки 
  * [Разведочный анализ данных](https://www.kaggle.com/emstrakhov/eda-with-pandas).<br>
  * [Моем датасет](https://proglib.io/p/moem-dataset-rukovodstvo-po-ochistke-dannyh-v-python-2020-03-27).<br>
  * Range Constraints: числа или даты должны попадать в определенный диапазон с использованием одной единицы измерения
  * Mandatory Constraints: некоторые столбцы не могут быть пустыми
  * Шаблоны регулярных выражений: текстовые поля, которые должны быть в определенном шаблоне. Например (999) 999–9999
  * Consistency: степень соответствия данных в одном или нескольких БД
  * [Фреймы в pandas](https://habr.com/ru/company/ruvds/blog/494720/)<br>
  * [Открытие фрейма в pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)<br> 

In [ ]:
from sklearn import preprocessing
import warnings
import numpy as np
import pandas as pd
import plotly
from IPython.display import Image
import matplotlib
from IPython.core.interactiveshell import InteractiveShell
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
import chart_studio.plotly as py
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
%matplotlib inline
matplotlib.style.use('ggplot')
warnings.filterwarnings("ignore")

InteractiveShell.ast_node_interactivity = 'all'


pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.width', 85)
pd.set_option('display.max_columns', 8)


# Рабочие фреймы
df = pd.read_csv('train.csv')
tsd = pd.read_csv('test.csv')
# Объединенный список из фреймов
comb = pd.concat([df, tsd], axis=0)

In [ ]:
# Описываем неоходимые манипуляции с колонками: смена типов, вид чистки, объединение, заполняемость NaN

In [ ]:
#Удаляем столбцы, где данные непрочитались
df = df.filter(regex='^(?!.*Unnamed).*$')

In [ ]:
#Сортировка числовых колонок
df['data'].value_counts(normalize=True)

In [ ]:
#Чистка от пустых значений
df = df.replace(-9999, np.nan)
print('Rows in the data frame: {0}'.format(len(df)))
print('Rows without NAN: {0}'.format(len(df.dropna(how='any'))))

In [ ]:
#Получение сокращенного фрейма по конкретным критериям
df[df["Sex"].isin([True])].head()

In [ ]:
# Преобразуем строчки с датами в объект datetime
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y') # format показывает что читаем: месяц, деньб год 

In [ ]:
#Построим график проверить тип тренда (линейный или нет), тип сезонности (аддитивный или мультипликативный), длину, выбросы
#Видим линейный тренд и мультипликативную сезонность. Это подтверждается после логирафмирование цикла 

fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(121)
df['date'].plot(ax=ax1)
ax1.set_title(u'коммент')
ax1.set_ylabel(u'коммент')

ax2 = fig.add_subplot(122)
pd.Series(np.log10(df['date'])).plot(ax=ax2)
ax2.set_title(u'коммент')
ax2.set_ylabel(u'коммент')

In [ ]:
#Shapiro-Wilk позволяют проверить выборку на принадлежность к ГС и нормальность распредеелния
df = df.set_index(u'номер')
plt.hist(np.log10(df[u'номер']), bins=50) #Применяем критерий Шапиро-Вилка после логарифмирования
res = stats.shapiro(np.log10(df[u'номер']))
print('p-value: ', res[1]) #Если p маленькое, гипотезу о нормальности отвергаем

In [ ]:
#Работа с выбросами. Точечное сокращение и усеченное среднее
df = df.iloc[2:1004] #Обрезать аномалии
exclude = int(len(df)/100*2.5) #Усеченное среднее (Среднее -2,5% самых малых и -2,5% наибольших)
df = df[exclude:len(df)-exclude]

In [ ]:
#Удаляем лишние знаки из значений объекта в колонке
df['data'] = df['data'].str.replace("\ ^ m \ $",'')

#Исправление опечаток
df['data'].map({'m': 'male', 'fem.': 'female', ...})
re.sub(r"\ ^ m \ $", ' Male ', ' male ', flags = re.IGNORECASE )

In [ ]:
#Заполнить пробелы Средним значением
rand = np.random.randint(average_age - 2*std_age, average_age + 2*std_age, size = count_nan_age) 
df["data"][np.isnan(df["data"])] = rand

In [ ]:
#Заполнение пропущенных значений функцией SimpleImputer
num_feat.remove("y")
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='median') #Выбор стратегии
imp.fit(comb[num_feat]) #Вычисление значений
comb[num_feat] = imp.transform(comb[num_feat]) #Применение стратегии

#Логарифмирование числовых колонок
for col in comb[num_feat]:
    comb[col] = comb[col].apply(lambda x: np.log1p(x))

num_feat.append("y")